In [ ]:
import numpy as np
import torch
import pandas as pd
from tensorflow.keras import regularizers
import json, requests
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Activation
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score, log_loss

In [ ]:
# !pip install -U sentence-transformers

In [ ]:
# bert_model = SentenceTransformer('xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
# bert_model = SentenceTransformer("xlm-roberta-base")

In [ ]:
# def get_embeddings(sentences):
#   sentence_embeddings = bert_model.encode(sentences, convert_to_tensor=True)
#   return tf.constant(sentence_embeddings, dtype=tf.float32)

In [ ]:
import tensorflow_hub as hub
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")


In [ ]:
def get_embeddings(sentences):
  sentence_embeddings = use_model(sentences)

  return sentence_embeddings

In [ ]:
# data = pd.read_csv('./drive/My Drive/colab/train.csv')

In [ ]:
# test_data = pd.read_csv('./drive/My Drive/colab/test.csv')

In [ ]:
train_data = pd.read_csv('./drive/My Drive/colab/training-nodwin''.csv')
# test_data = pd.read_csv('./drive/My Drive/colab/dataset/insurance_test_cleaned''.csv')

In [ ]:
# data = train_data[train_data.DELETE != 'TRUE']

In [ ]:
import time

start = time.time()
X_train = get_embeddings(train_data['Utterance'])
print("time taken = ", time.time()-start)

time taken =  6.424519300460815


In [ ]:
def precision(y_true, y_pred):
    """ Computes the F score, the weighted harmonic mean of precision and recall.
        Calculates the precision, a metric for multi-label classification of
        how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + K.epsilon())


def recall(y_true, y_pred):
    """
        Calculates the recall, a metric for multi-label classification of
        how many relevant items are selected.

    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())




def on_epoch_end(x,y):
    print('x test shape',x.shape)
    print(' test shape',x.shape)
    loss, acc = model.evaluate(x, y, verbose=0)
    print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))
    return loss


def train_embeddings_v2(x_train, y_train, C=0, epochs=70):
    # computing the class weights here
    y_integers = np.argmax(y_train, axis=1)
    class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
    d_class_weights = dict(enumerate(class_weights))
    if len(d_class_weights) <= 1:
        d_class_weights = 'auto'
    rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-6, mode='min', verbose=1)
    adam = Adam(learning_rate=1e-2)
    print("Starting training with regularization " + str(C))
    model = Sequential()
    model.add(Dense(x_train.shape[1], input_shape=(x_train.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dense(y_train.shape[1]))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy', precision, recall])
    model.fit(x_train, y_train, epochs=epochs, batch_size=32, callbacks=[rlr],
              class_weight=d_class_weights, shuffle=False)
    return model

def predict(text):
  X_test = get_embeddings([text])
  y_pred = model.predict(X_test)
  y_pred = list(np.argmax(y_pred,axis=1))
  y_pred_lables = list(label_encoder.inverse_transform(y_pred))
  return y_pred_lables





In [ ]:
def predict_debug(text):
  X_test = get_embeddings([text])
  y_pred = model.predict(X_test)
  y_pred_pos = list(np.argmax(y_pred,axis=1))
  y_pred_lables = label_encoder.inverse_transform(y_pred_pos)
  
  return y_pred_lables[0],y_pred[0][y_pred_pos][0]
  
  print(y_pred[0][y_pred_pos])                     
  if(y_pred[0][y_pred_pos]<0.6):
    return -1
  return y_pred_lables

In [ ]:
y_train_labels  =  (train_data['Intent']).tolist()

In [ ]:
# negative_embeddings = np.load('./drive/My Drive/colab/xlm_gib.npz')['embeddings']

In [ ]:
# Adding negative embeddings

# cos_sim = cosine_similarity(X_train, negative_embeddings)
# r, c = np.where(cos_sim >= 0.7)
# negative_embeddings = np.delete(negative_embeddings, list(set(c)), axis=0)
# X_train = np.concatenate((X_train, negative_embeddings), axis=0)
# y_train_labels = np.append(y_train_labels, np.full(len(negative_embeddings), "none"))

In [ ]:
# X_train = np.load("./drive/My Drive/colab/x_t.npy")
# y_train_labels = np.load("./drive/My Drive/colab/y_t.npy")

In [ ]:
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False, dtype=np.int32)
integer_encoded = label_encoder.fit(y_train_labels)
integer_encoded_train = integer_encoded.transform(y_train_labels)
integer_encoded_train = integer_encoded_train.reshape(len(integer_encoded_train), 1)
onehot_encoded_intents_train = onehot_encoder.fit_transform(integer_encoded_train)
y_train = np.array(onehot_encoded_intents_train)
model = train_embeddings_v2(X_train, y_train, C=0,epochs=25)

Starting training with regularization 0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 18)                9234      
_________________________________________________________________
activation_1 (Activation)    (None, 18)                0         
Total params: 271,890
Trainable params: 271,890
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Epoch 1/25
7/7 [==============================] - 1s 5ms/step - loss: 0.3822 - accuracy: 0.2398 - precision: 0.0055 - recall: 0.0446
Epoch 2/25
7/7 [==============================] - 0s 4ms/step - loss: 0.2384 - accuracy: 0.6582 - precision: 0.1429 - recall: 0.1071    
Epoch 3/25
7/7 [==============================] - 0s 4ms/step - loss: 0.0863 - accuracy: 0.9184 - precision: 0.9577 - recall: 0.6295
Epoch 4/25
7/7 [==============================] - 0s 5ms/step - loss: 0.0387 - accuracy: 0.9745 - precision: 0.9744 - recall: 0.9196
Epoch 5/25
7/7 [==============================] - 0s 6ms/step - loss: 0.0191 - accuracy: 0.9898 - precision: 0.9874 - recall: 0.9107
Epoch 6/25
7/7 [==============================] - 0s 4ms/step - loss: 0.0088 - accuracy: 0.9898 - precision: 0.9833 - recall: 0.9955
Epoch 7/25
7/7 [==============================] - 0s 5ms/step - loss: 0.0051 - accuracy: 0.9898 - precision: 0.9911 - recall: 0.9866
Epoch 8/25
7/7 [==============================] - 0s 4ms/step - l

In [ ]:
start = time.time()
predict("My card has not arrived yet.")

print("Time = ",time.time()-start)

Time =  0.38132572174072266


In [ ]:
# import csv
# with open('./drive/My Drive/colab/result/use_result.csv', 'w') as csvfile: 
#   csvwriter = csv.writer(csvfile) 
#   csvwriter.writerow(['UTTERANCES','INTENT','PREDICTION','CONFIDENCE'])
#   for i in range(1,len(test_data)):
#     print(i, end=" ")
#     pred_res = predict_debug(test_data["UTTERANCES"][i])
#     csvwriter.writerow([test_data["UTTERANCES"][i],test_data["INTENT_NAME"][i],pred_res[0],pred_res[1]])
    


5871 5872 5873 5874 5875 5876 5877 5878 5879 5880 5881 5882 5883 5884 5885 5886 5887 5888 5889 5890 5891 5892 5893 5894 5895 5896 5897 5898 5899 5900 5901 5902 5903 5904 5905 5906 5907 5908 5909 5910 5911 5912 5913 5914 5915 5916 5917 5918 5919 5920 5921 5922 5923 5924 5925 5926 5927 5928 5929 5930 5931 5932 5933 5934 5935 5936 5937 5938 5939 5940 5941 5942 5943 5944 5945 5946 5947 5948 5949 5950 5951 5952 5953 5954 5955 5956 5957 5958 5959 5960 5961 5962 5963 5964 5965 5966 5967 5968 5969 5970 5971 5972 5973 5974 5975 5976 5977 5978 5979 5980 5981 5982 5983 5984 5985 5986 5987 5988 5989 5990 5991 5992 5993 5994 5995 5996 5997 5998 5999 6000 6001 6002 6003 6004 6005 6006 6007 6008 6009 6010 6011 6012 6013 6014 6015 6016 6017 6018 6019 6020 6021 6022 6023 6024 6025 6026 6027 6028 6029 6030 6031 6032 6033 6034 6035 6036 6037 6038 6039 6040 6041 6042 6043 6044 6045 6046 6047 6048 6049 6050 6051 6052 6053 6054 6055 6056 6057 6058 6059 6060 6061 6062 6063 6064 6065 6066 6067 6068 6069 6070 

In [ ]:
right = 0
wrong = 0
no_answer = 0
for i in range(1,len(test_data)):
  # print(i,end=" ")
  pred_res = predict_debug(test_data["Utterance"][i])
  if pred_res[1]<0.7:
    print(i, " " , test_data["Utterance"][i], ": " ,test_data["Intent"][i])
    print(pred_res)
    no_answer+=1
  elif pred_res[0] == test_data["Intent"][i]:
    
    right+=1
  else:
    print(i, " " , test_data["Utterance"][i], ": " ,test_data["Intent"][i])
    print(pred_res)
    wrong+=1

4   How long will it take to add a person in my existing policy :  INFO_ADD_REMOVE_INSURED
('INFO_ADD_REMOVE_INSURED', 0.4951897)
5   I was printing my cards off an realized my husbands name was not on them I need to make sure he is insured also :  INFO_ADD_REMOVE_INSURED
('getting_spare_card', 0.81892097)
8   remove sam from my policy :  INFO_ADD_REMOVE_INSURED
('INFO_CANCEL_CONFIRM', 0.999951)
16   This is my third attempt to remove my deceased wifes name from the account I would appreciate it if this was finally taken care of :  INFO_ADD_REMOVE_INSURED
('INFO_CANCEL_CONFIRM', 0.041423798)
18   delete colin from my policy :  INFO_ADD_REMOVE_INSURED
('INFO_CANCEL_CONFIRM', 1.0)
25   If my coverage is covering anyone but myself they need to be deleted :  INFO_ADD_REMOVE_INSURED
('INFO_CANCEL_CONFIRM', 0.99999785)
26   If someone else drives my car how do I get them on the account :  INFO_ADD_REMOVE_INSURED
('INFO_ADD_REMOVE_INSURED', 0.45592132)
30   How do I get names removed from my 

In [ ]:
print(right,wrong,no_answer)

5811 1002 712


In [ ]:
predictions = []
for text in test_data["Utterance"]:
  predictions.append(predict(text))

KeyError: ignored

In [ ]:
len(predictions)

In [ ]:
accuracy_score(test_data['Intent'], predictions)

('card_delivery_estimate', 0.06717551)

In [ ]:
import os
os.environ['PYTHONHASHSEED']=str(10)
import random
random.seed(10)

np.random.seed(10)
tf.random.set_seed(10)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False, dtype=np.int32)
integer_encoded = label_encoder.fit(y_train_labels)
integer_encoded_train = integer_encoded.transform(y_train_labels)
integer_encoded_train = integer_encoded_train.reshape(len(integer_encoded_train), 1)
onehot_encoded_intents_train = onehot_encoder.fit_transform(integer_encoded_train)
y_train = np.array(onehot_encoded_intents_train)
model = train_embeddings_v2(X_train, y_train, C=0,epochs=15)

Starting training with regularization 0
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 768)               590592    
_________________________________________________________________
activation (Activation)      (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 78)                59982     
_________________________________________________________________
activation_1 (Activation)    (None, 78)                0         
Total params: 650,574
Trainable params: 650,574
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
329/329 [==============================] - 3s 8ms/step - loss: 0.1321 - accuracy: 0.0586 - precision: 0.0598 - recall: 0.0632
Epoch 2/15
329/329 [==============================] - 3s 8ms/step - los

In [ ]:
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False, dtype=np.int32)
integer_encoded = label_encoder.fit(y_train_labels)
integer_encoded_train = integer_encoded.transform(y_train_labels)
integer_encoded_train = integer_encoded_train.reshape(len(integer_encoded_train), 1)
onehot_encoded_intents_train = onehot_encoder.fit_transform(integer_encoded_train)
y_train = np.array(onehot_encoded_intents_train)
model = train_embeddings_v2(X_train, y_train, C=0,epochs=15)

Starting training with regularization 0
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 768)               590592    
_________________________________________________________________
activation_2 (Activation)    (None, 768)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 78)                59982     
_________________________________________________________________
activation_3 (Activation)    (None, 78)                0         
Total params: 650,574
Trainable params: 650,574
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
329/329 [==============================] - 3s 10ms/step - loss: 0.1353 - accuracy: 0.0617 - precision: 0.0663 - recall: 0.0676
Epoch 2/15
329/329 [==============================] - 4s 11ms/step -

In [ ]:
predict_debug("My card has not arrived yet")